In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Import library files

In [7]:
import torch
from tqdm.notebook import tqdm
import numpy as np 
import pandas as pd

Decode and clean the dataset

In [8]:
import json

data = []
#for line in open('/content/test.json', 'r'):
  #data.append(json.loads(line))
# Opening JSON file
f = open("/content/drive/MyDrive/sylver/News_Category_Dataset_v3.json")
 
# Decoding : returns JSON object as
# a dictionary
data = json.load(f)
#data
 
df = pd.DataFrame.from_dict(data)
df.head(5)
df=df.drop(['link', 'authors','date'], axis=1)
#df2 = pd.DataFrame()
df["Final"] = df['headline'] +" "+ df["short_description"]
df.info()
df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 209527 entries, 0 to 209526
Data columns (total 4 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   headline           209527 non-null  object
 1   category           209527 non-null  object
 2   short_description  209527 non-null  object
 3   Final              209527 non-null  object
dtypes: object(4)
memory usage: 6.4+ MB


,headline,category,short_description,Final
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,Over 4 Million Americans Roll Up Sleeves For O...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,"American Airlines Flyer Charged, Banned For Li..."
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",23 Of The Funniest Tweets About Cats And Dogs ...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",The Funniest Tweets From Parents This Week (Se...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Woman Who Called Cops On Black Bird-Watcher Lo...


Exploratory Data Analysis and Preprocessing

In [9]:
df.head()

,headline,category,short_description,Final
0,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,Over 4 Million Americans Roll Up Sleeves For O...
1,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,"American Airlines Flyer Charged, Banned For Li..."
2,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",23 Of The Funniest Tweets About Cats And Dogs ...
3,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",The Funniest Tweets From Parents This Week (Se...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Woman Who Called Cops On Black Bird-Watcher Lo...


In [10]:
df.category.value_counts()
#df.label_bias.value_counts()

POLITICS          35602
WELLNESS          17945
ENTERTAINMENT     17362
TRAVEL             9900
STYLE & BEAUTY     9814
PARENTING          8791
HEALTHY LIVING     6694
QUEER VOICES       6347
FOOD & DRINK       6340
BUSINESS           5992
COMEDY             5400
SPORTS             5077
BLACK VOICES       4583
HOME & LIVING      4320
PARENTS            3955
THE WORLDPOST      3664
WEDDINGS           3653
WOMEN              3572
CRIME              3562
IMPACT             3484
DIVORCE            3426
WORLD NEWS         3299
MEDIA              2944
WEIRD NEWS         2777
GREEN              2622
WORLDPOST          2579
RELIGION           2577
STYLE              2254
SCIENCE            2206
TECH               2104
TASTE              2096
MONEY              1756
ARTS               1509
ENVIRONMENT        1444
FIFTY              1401
GOOD NEWS          1398
U.S. NEWS          1377
ARTS & CULTURE     1339
COLLEGE            1144
LATINO VOICES      1130
CULTURE & ARTS     1074
EDUCATION       

In [11]:
possible_labels = df.category.unique()
#possible_labels = df.label_bias.unique()


In [12]:
possible_labels


array(['U.S. NEWS', 'COMEDY', 'PARENTING', 'WORLD NEWS', 'CULTURE & ARTS',
       'TECH', 'SPORTS', 'ENTERTAINMENT', 'POLITICS', 'WEIRD NEWS',
       'ENVIRONMENT', 'EDUCATION', 'CRIME', 'SCIENCE', 'WELLNESS',
       'BUSINESS', 'STYLE & BEAUTY', 'FOOD & DRINK', 'MEDIA',
       'QUEER VOICES', 'HOME & LIVING', 'WOMEN', 'BLACK VOICES', 'TRAVEL',
       'MONEY', 'RELIGION', 'LATINO VOICES', 'IMPACT', 'WEDDINGS',
       'COLLEGE', 'PARENTS', 'ARTS & CULTURE', 'STYLE', 'GREEN', 'TASTE',
       'HEALTHY LIVING', 'THE WORLDPOST', 'GOOD NEWS', 'WORLDPOST',
       'FIFTY', 'ARTS', 'DIVORCE'], dtype=object)

In [13]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [14]:
label_dict

{'U.S. NEWS': 0,
 'COMEDY': 1,
 'PARENTING': 2,
 'WORLD NEWS': 3,
 'CULTURE & ARTS': 4,
 'TECH': 5,
 'SPORTS': 6,
 'ENTERTAINMENT': 7,
 'POLITICS': 8,
 'WEIRD NEWS': 9,
 'ENVIRONMENT': 10,
 'EDUCATION': 11,
 'CRIME': 12,
 'SCIENCE': 13,
 'WELLNESS': 14,
 'BUSINESS': 15,
 'STYLE & BEAUTY': 16,
 'FOOD & DRINK': 17,
 'MEDIA': 18,
 'QUEER VOICES': 19,
 'HOME & LIVING': 20,
 'WOMEN': 21,
 'BLACK VOICES': 22,
 'TRAVEL': 23,
 'MONEY': 24,
 'RELIGION': 25,
 'LATINO VOICES': 26,
 'IMPACT': 27,
 'WEDDINGS': 28,
 'COLLEGE': 29,
 'PARENTS': 30,
 'ARTS & CULTURE': 31,
 'STYLE': 32,
 'GREEN': 33,
 'TASTE': 34,
 'HEALTHY LIVING': 35,
 'THE WORLDPOST': 36,
 'GOOD NEWS': 37,
 'WORLDPOST': 38,
 'FIFTY': 39,
 'ARTS': 40,
 'DIVORCE': 41}

In [15]:
df.category = df['category'].map(label_dict)


In [16]:
df.head(10)

,headline,category,short_description,Final
0,Over 4 Million Americans Roll Up Sleeves For O...,0,Health experts said it is too early to predict...,Over 4 Million Americans Roll Up Sleeves For O...
1,"American Airlines Flyer Charged, Banned For Li...",0,He was subdued by passengers and crew when he ...,"American Airlines Flyer Charged, Banned For Li..."
2,23 Of The Funniest Tweets About Cats And Dogs ...,1,"""Until you have a dog you don't understand wha...",23 Of The Funniest Tweets About Cats And Dogs ...
3,The Funniest Tweets From Parents This Week (Se...,2,"""Accidentally put grown-up toothpaste on my to...",The Funniest Tweets From Parents This Week (Se...
4,Woman Who Called Cops On Black Bird-Watcher Lo...,0,Amy Cooper accused investment firm Franklin Te...,Woman Who Called Cops On Black Bird-Watcher Lo...
5,Cleaner Was Dead In Belk Bathroom For 4 Days B...,0,The 63-year-old woman was seen working at the ...,Cleaner Was Dead In Belk Bathroom For 4 Days B...
6,Reporter Gets Adorable Surprise From Her Boyfr...,0,"""Who's that behind you?"" an anchor for New Yor...",Reporter Gets Adorable Surprise From Her Boyfr...
7,Puerto Ricans Desperate For Water After Hurric...,3,More than half a million people remained witho...,Puerto Ricans Desperate For Water After Hurric...
8,How A New Documentary Captures The Complexity ...,4,"In ""Mija,"" director Isabel Castro combined mus...",How A New Documentary Captures The Complexity ...
9,Biden At UN To Call Russian War An Affront To ...,3,White House officials say the crux of the pres...,Biden At UN To Call Russian War An Affront To ...


Classes are imbalanced as visible

## 2: Training/Validation Split

In [17]:
from sklearn.model_selection import train_test_split

In [18]:
X_train, X_val, y_train, y_val = train_test_split(df.index.values, 
                                                  df.category.values,
                                                  test_size=0.15, 
                                                  random_state=42,
                                                  stratify=df.category.values
                                                  #stratify=df.label_bias.values
                                                  )

In [19]:
df['data_type'] = ['not_set']*df.shape[0]

In [20]:
df.head()

,headline,category,short_description,Final,data_type
0,Over 4 Million Americans Roll Up Sleeves For O...,0,Health experts said it is too early to predict...,Over 4 Million Americans Roll Up Sleeves For O...,not_set
1,"American Airlines Flyer Charged, Banned For Li...",0,He was subdued by passengers and crew when he ...,"American Airlines Flyer Charged, Banned For Li...",not_set
2,23 Of The Funniest Tweets About Cats And Dogs ...,1,"""Until you have a dog you don't understand wha...",23 Of The Funniest Tweets About Cats And Dogs ...,not_set
3,The Funniest Tweets From Parents This Week (Se...,2,"""Accidentally put grown-up toothpaste on my to...",The Funniest Tweets From Parents This Week (Se...,not_set
4,Woman Who Called Cops On Black Bird-Watcher Lo...,0,Amy Cooper accused investment firm Franklin Te...,Woman Who Called Cops On Black Bird-Watcher Lo...,not_set


In [21]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [22]:
df.groupby(['category', 'data_type']).count()
#df.groupby(['label_bias', 'data_type']).count()

headline  short_description  Final
category data_type                                    
0        train          1171               1171   1171
         val             206                206    206
1        train          4590               4590   4590
         val             810                810    810
2        train          7472               7472   7472
...                      ...                ...    ...
39       val             210                210    210
40       train          1283               1283   1283
         val             226                226    226
41       train          2912               2912   2912
         val             514                514    514

[84 rows x 3 columns]

# 3. Loading Tokenizer and Encoding our Data

In [23]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [25]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

In [26]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].short_description.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].short_description.values,
    add_special_tokens=True,
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256,
    return_tensors='pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].category.values)
#labels_train = torch.tensor(df[df.data_type=='train'].label_bias.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].category.values)
#labels_val = torch.tensor(df[df.data_type=='val'].label_bias.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [27]:
dataset_train = TensorDataset(input_ids_train, 
                              attention_masks_train,
                              labels_train)

dataset_val = TensorDataset(input_ids_val, 
                            attention_masks_val,
                           labels_val)

In [28]:
len(dataset_train)

178097

In [29]:
dataset_val.tensors

(tensor([[  101,  2002,  2001,  ...,     0,     0,     0],
         [  101,  2062,  2084,  ...,     0,     0,     0],
         [  101,  2028,  2158,  ...,     0,     0,     0],
         ...,
         [  101,  1000, 18288,  ...,     0,     0,     0],
         [  101,  2002,  2001,  ...,     0,     0,     0],
         [  101,  2065,  2017,  ...,     0,     0,     0]]),
 tensor([[1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         ...,
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0],
         [1, 1, 1,  ..., 0, 0, 0]]),
 tensor([ 0,  3,  5,  ..., 15,  5,  5]))

# 4. Setting up BERT Pretrained Model

In [30]:
from transformers import BertForSequenceClassification

In [31]:
model = BertForSequenceClassification.from_pretrained(
                                      'bert-base-uncased', 
                                      num_labels = len(label_dict),
                                      output_attentions = False,
                                      output_hidden_states = False
                                     )

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

# 5. Creating Data Loaders

In [32]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [33]:
dataset_train

In [34]:
batch_size = 4

dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

# 6. Setting Up Optimizer and Scheduler

In [35]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [36]:
optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

In [37]:
epochs = 1

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps = len(dataloader_train)*epochs
)

# 7. Defining our Performance Metrics

In [38]:
import numpy as np
from sklearn.metrics import f1_score

In [39]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average = 'weighted')

In [40]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat==label]
        y_true = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy:{len(y_preds[y_preds==label])}/{len(y_true)}\n')

# 8. Creating our Training Loop

In [41]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
print(device)

cuda


In [43]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [44]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train, 
                        desc='Epoch {:1d}'.format(epoch), 
                        leave=False, 
                        disable=False)
    
    for batch in progress_bar:
        model.zero_grad()
        batch = tuple(b.to(device) for b in batch)
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss_train_total +=loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})     
    
    torch.save(model, f'BERT_ft_Epoch{epoch}.model')
    
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1:   0%|          | 0/44525 [00:00<?, ?it/s]


Epoch 1
Training loss: 1.9055079322717017


  0%|          | 0/983 [00:00<?, ?it/s]

Validation loss: 1.7199934194071005
F1 Score (weighted): 0.4926876847174895


In [45]:
###EVALUATION

**###EVALUATION**

In [46]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [47]:
import warnings
from transformers import BertTokenizer
import torch
from transformers import BertForSequenceClassification
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings('ignore')
#{'Non-biased': 0, 'Biased': 1}

labels = {
 0:'U.S. NEWS',
 1:'COMEDY',
 2:'PARENTING',
 3:'WORLD NEWS',
 4:'CULTURE & ARTS',
 5: "TECH",
 6: "SPORTS",
 7:'ENTERTAINMENT',
 8:'POLITICS',
 9:'WEIRD NEWS',
 10:'ENVIRONMENT',
 11:'EDUCATION',
 12:'CRIME',
 13:'SCIENCE',
 14:'WELLNESS',
 15:'BUSINESS',
 16:'STYLE & BEAUTY',
 17:'FOOD & DRINK',
 18:'MEDIA',
 19:'QUEER VOICES',
 20:'HOME & LIVING',
 21:'WOMEN',
 22:'BLACK VOICES',
 23:'TRAVEL',
 24:'MONEY',
 25:'RELIGION',
 26:'LATINO VOICES',
 27:'IMPACT',
 28:'WEDDINGS',
 29:'COLLEGE',
 30:'PARENTS',
 31:'ARTS & CULTURE',
 32:'STYLE',
 33:'GREEN',
 34:'TASTE',
 35:'HEALTHY LIVING',
 36:'THE WORLDPOST',
 37:'GOOD NEWS',
 38:'WORLDPOST',
 39:'FIFTY',
 40:'ARTS',
 41:'DIVORCE'
       }
tokenizer = BertTokenizer.from_pretrained(
'bert-base-uncased',
do_lower_case=True
)






In [48]:
    
def predict_bias(NewsHeadine):
    model = torch.load('/content/BERT_ft_Epoch1.model',map_location=torch.device('cpu'))
    device = torch.device('cpu')
    print(device)
    encoded_review = tokenizer(NewsHeadine, return_tensors="pt")

    input_ids = encoded_review['input_ids'].to(device)
    attention_mask = encoded_review['attention_mask'].to(device)
    output = model(input_ids, attention_mask)
    final_pred = torch.tensor(output.logits)
    categoryIndex = int(torch.argmax(final_pred))
    category = labels[categoryIndex]
    return category


In [49]:
NewsHeadine = "Charter school advocates are well-versed in their talking points, but those points are often devoid of fact."
Category = predict_bias(NewsHeadine)
print(Category)


cpu
EDUCATION
